In [ ]:
!pip install -q pyomo
!apt-get install -y -qq coinor-cbc

In [2]:
from pyomo.environ import *
import numpy as np
import pandas as pd


### 1)

Let 
P = [175000, 150000, 125000,  90000,  80000, 120000]\
E = [250, 400, 450, 100, 200,  50]\
L = [1800, 2800, 3500,  800,  700,  500]

$Let P_i, L_i, E_i\ \  \text{denote the i-th element of  P, L and E respectively}$

The problem can be formulated as:

Decision variables: $b_0,\ b_1,\ b_2$\
$minimize \quad \sum_{i=1}^{6} |P_i - (b_0 + b_1L_i + b_2E_i)|$\
S.T\
$ b_0 \geq 0$


The above optimisation problem is not linear, since it has modulus terms in the objective function

In [4]:
df = pd.read_csv('ex3.csv',index_col=0)

In [5]:
n = 4

pri = df[' Pi'].values
lot = df['Li'].values
elev = df['Ei'].values

### Approach 1

The problem can be reformulated as:

Decision variables: 
$b_0,\ b_1,\ b_2\\
z_i \qquad i = 1,2,3,4,5,6$\


$minimize \quad \sum_{i=1}^{6} z_i$\
S.T\
$z_i \geq P_i - (b_0 + b_1L_i + b_2E_i) \qquad i = 1,2,3,4,5,6\\
z_i \geq -(P_i - (b_0 + b_1L_i + b_2E_i)) \qquad i = 1,2,3,4,5,6\\
b_0 \geq 0$


In [6]:
# Approach 1

m3 = ConcreteModel()
m3.b = Var(np.arange(3))
m3.z = Var(np.arange(6),domain=NonNegativeReals)
m3.b[0].setlb(0)

m3.cons = ConstraintList()
for i in range(6):
    m3.cons.add(m3.z[i]>= m3.b[0] + lot[i]*m3.b[1]+ elev[i]*m3.b[2] - pri[i])
    m3.cons.add(-m3.z[i]<= m3.b[0] + lot[i]*m3.b[1]+ elev[i]*m3.b[2] - pri[i])

m3.res = Objective(expr = summation(m3.z))    

### Approach 2

The problem can be reformulated as:

Decision variables: 
$b_0,\ b_1,\ b_2\\
x_i, y_i \qquad i = 1,2,3,4,5,6$\


$minimize \quad \sum_{i=1}^{6} (x_i + y_i)$\
S.T\
$x_i - yi = P_i - (b_0 + b_1L_i + b_2E_i) \qquad i = 1,2,3,4,5,6\\
b_0 \geq 0$

In [7]:
# Approach 2

m32 = ConcreteModel()
m32.b = Var(np.arange(3))
m32.b[0].setlb(0)
m32.x = Var(np.arange(6),domain=NonNegativeReals)
m32.y = Var(np.arange(6),domain=NonNegativeReals)
m32.cons = ConstraintList()

m32.res = Objective(expr=summation(m32.x)+summation(m32.y))



for i in range(6):
    m32.cons.add(m32.x[i]-m32.y[i]== m32.b[0] + lot[i]*m32.b[1]+ elev[i]*m32.b[2] - pri[i])

In [8]:
solver = SolverFactory('cbc')

### 5)

In [9]:
#Approach 1
opt1 = solver.solve(m3)
print('Solver status:', opt1.solver.status)
print('Solver termination condition:',opt1.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [10]:
# display solution
print('\nOptimal value of sum of residuals is = ', m3.res())

print('\nDecision Variables:')
for i in range(3):
    print('Value of b%d is'%i, m3.b[i].value)

for j in range(6):
    print('Value of z%d is'%(j+1), m3.z[j].value)


Optimal value of sum of residuals is =  125454.545

Decision Variables:
Value of b0 is 104181.82
Value of b1 is 67.272727
Value of b2 is -356.36364
Value of z1 is 38818.182
Value of z2 is 0.0
Value of z3 is 54272.727
Value of z4 is 32363.636
Value of z5 is 0.0
Value of z6 is 0.0


In [11]:
#Approach 2

opt2 = solver.solve(m32)
print('Solver status:',opt2.solver.status)
print('Solver termination condition:',opt2.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [12]:
print('\nOptimal value of sum of residuals is = ', m32.res())

print('\nDecision Variables:')
for i in range(3):
    print('Value of b%d is'%i, m32.b[i].value)
print('\n')
for j in range(6):
    print('Value of x%d is'%(j+1), m32.x[j].value)
    print('Value of y%d is'%(j+1), m32.y[j].value)


Optimal value of sum of residuals is =  125454.545

Decision Variables:
Value of b0 is 104181.82
Value of b1 is 67.272727
Value of b2 is -356.36364


Value of x1 is 0.0
Value of y1 is 38818.182
Value of x2 is 0.0
Value of y2 is 0.0
Value of x3 is 54272.727
Value of y3 is 0.0
Value of x4 is 32363.636
Value of y4 is 0.0
Value of x5 is 0.0
Value of y5 is 0.0
Value of x6 is 0.0
Value of y6 is 0.0


Both methods give the same solution. This shows that they are equivalent. 